In [1]:
from mars.db import collections
from mars.db import db_fields, fetch_batches_until_empty
# from mars.sentences_splitting import split_to_sentences_sentences

from mars.parser import parse_documents
from mars.segmentation.segmentation import segment_and_upload
from mars import config

In [4]:
docs = collections.document_sources.fetchByExample(
    {"dataset": "DocumentLevelDataset.jobin2019"}, 1000
)

In [5]:
for doc in docs:
    ...
doc['_key']

In [7]:
doc

ArangoDoc '_id: Documents/32568617, _key: 32568617, _rev: _dXo9inO---': <store: {'url': 'https://www.article19.org/wp-content/uploads/2018/04/Privacy-and-Freedom-of-Expression-In-the-Age-of-Artificial-Intelligence-1.pdf', 'file_type': 'html', 'filename': '8fc7e0ae-b912-4e6e-a49e-ce9bff7505bf.html', 'source_website': 'manually_added', 'user': 'jakwisn', 'name': None, 'jobs': [], 'dataset': 'DocumentLevelDataset.jobin2019'}>

In [7]:
key = int(doc['_key'])
segment_and_upload(key,key)

12/08/2021 05:07:18 Segment documents from 32568617 to 32568617
12/08/2021 05:07:47 Already segmented documents: 0 / 1
12/08/2021 05:07:47 Waiting for segmentation documents: 1 / 1
12/08/2021 05:07:47 Segmenting Documents/32568617 (0.0%)
/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/mars/segmentation/html_segmantation.py:35: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 35 of the file /home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/mars/segmentation/html_segmantation.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(top_node)

12/08/2021 05:07:49 Parse time: 1s | Save tim

In [23]:
d = collections.segmented_texts.fetchByExample({'source_doc_id':'Documents/32568617'},1000)
for s in d:
    print("---")
    print(s.content)

---
Privacy and Freedom of Expression in the Age of Artificial Intelligence
---
1/29
---
Privacy and Freedom of Expression 
---
In the Age of Artificial Intelligence
---
April 2018


In [20]:
len(d)

0

In [14]:
for a in d:
    print(a.content)

Privacy and Freedom of Expression in the Age of Artificial Intelligence
1/29
Privacy and Freedom of Expression 
In the Age of Artificial Intelligence
April 2018


In [11]:
len(collections.sentences.fetchByExample({db_fields.DOC_ID:doc['_id']},1000))

0

In [10]:
sent_docs = [d for d in collections.sentences.fetchByExample({db_fields.DOC_ID:doc['_id']},1000)]

===
Unable to establish connection, perhaps arango is not running.
===


KeyboardInterrupt: 

In [10]:
doc['_id']

'Documents/26774830'

In [15]:
sent_docs = collections.sentences.fetchByExample({db_fields.DOC_ID:doc['_id']},1000)

In [17]:
len(sent_docs)

0

In [13]:
collections.sentences.ensurePersistentIndex(['source_doc_id','sequence_number','sentence_number'],unique=True)
collections.sentences.ensurePersistentIndex(['source_doc_id'])

<Index of type persistent>

<Index of type persistent>

# Migrate to new structure

In [11]:
for i, batch in enumerate(
    fetch_batches_until_empty(
        """FOR x in Sentences FILTER not IS_ARRAY(x.embedding) LIMIT 10 RETURN x """
    )
):
    print("processing batch:", i)
    for a in batch:
        emb = a[db_fields.EMBEDDING]
        a[db_fields.EMBEDDING] = dict()
        a[db_fields.EMBEDDING][db_fields.EmbeddingType.LABSE] = emb
        to_save.append(a)
    collections.sentences.bulkSave(to_save, onDuplicate="update")

In [22]:
for i, batch in enumerate(
    fetch_batches_until_empty(
        """FOR x in Sentences FILTER IS_ARRAY(x.embedding) LIMIT 1000 RETURN x """
    )
):
    print("processing batch:", i)
    to_save=[]
    for a in batch:
        emb = a[db_fields.EMBEDDING]
        a[db_fields.EMBEDDING] = dict()
        a[db_fields.EMBEDDING][db_fields.EmbeddingType.LABSE] = emb
        to_save.append(a)
    collections.sentences.bulkSave(to_save, onDuplicate="update")

processing batch: 0
processing batch: 1
processing batch: 2
processing batch: 3
processing batch: 4
processing batch: 5
processing batch: 6
processing batch: 7
processing batch: 8
processing batch: 9
processing batch: 10
processing batch: 11
processing batch: 12
processing batch: 13
processing batch: 14
processing batch: 15
processing batch: 16
processing batch: 17
processing batch: 18
processing batch: 19
processing batch: 20
processing batch: 21
processing batch: 22
processing batch: 23
processing batch: 24
processing batch: 25
processing batch: 26
processing batch: 27
processing batch: 28
processing batch: 29
processing batch: 30
processing batch: 31
processing batch: 32
processing batch: 33
processing batch: 34
processing batch: 35
processing batch: 36
processing batch: 37
processing batch: 38
processing batch: 39
processing batch: 40
processing batch: 41
processing batch: 42
processing batch: 43
processing batch: 44
processing batch: 45
processing batch: 46
processing batch: 47
pr

KeyboardInterrupt: 

In [16]:
a[0]

ArangoDoc '_id: Sentences/18549982, _key: 18549982, _rev: _dJ4tfiS---': <store: {'embedding': ['0.036199465', '-0.018653592', '0.05235075', '0.009604546', '0.06969898', '0.046939425', '-0.02927087', '0.019418977', '-0.00017126776', '-0.014687742', '0.04624988', '-0.027533641', '0.064110056', '0.0025424773', '0.013881902', '-0.08221822', '0.03419351', '0.038236335', '0.019841185', '0.017139452', '0.006993081', '0.0020378262', '0.013722185', '0.009196143', '-0.03146953', '-0.025850048', '-0.04201935', '-0.06952162', '0.020262875', '-0.052432932', '-0.06429708', '-0.008869282', '0.048512552', '0.0112873465', '-0.0597143', '-0.025724897', '-0.0065024663', '0.014398951', '-0.08008629', '-0.018840007', '-0.02831749', '0.049758203', '-0.019606791', '-0.04002189', '-0.041223098', '0.04035436', '0.016163109', '-0.0029556525', '0.016940083', '0.08025375', '0.065412484', '-0.04809125', '-0.030743117', '0.024526512', '0.006988894', '0.012589923', '-0.0070291287', '-0.008746413', '-0.04212336', '-0